# Train Model for [Target]

## 1.Imports

In [ ]:
import autoreload
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
frm src import utils, write, version_name
from src.modelling import prep, train, visualisations as viz

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from xgboost import XBGRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from lazypredict.Supervised import LazyRegressor
from sklearn.metrics import r2_score

## 2. Review Config

### Google Cloud Storage

In [ ]:
utils.read_config('gcs_paths.yaml')

### Model Config

In [ ]:
model_config = {
    'wtr_whitespace': False,
    'jl_whitespace': False,
    'print_viz': 'nb'
}
write.dict_to_yaml(model_config, 'model_config.yaml', base=False)

## Set Target Variable

In [ ]:
utils.read_config('target_variables.yaml')  # list all possible target variables.

In [ ]:
write.dict_to_yaml(
    {'targets': ['jl_branch_sales']},
    'target_variables.yaml',
    base=False
    )

## Get Training Data, Features and Targets

In [ ]:
target = utils.read_config('target_variables.yaml')['targets'][0]
target

In [ ]:
df_at = prep.import_analytics_table()
features = prep.get_features()
n_features = len(features)
n_features

## Feature Selection

### Feature Importance

In [ ]:
tree_params = {
    'max_depth': -1  # growing an unpruned tree for feature importances
    'colsample+bytree': 0.2,  # consider only a smaller number of features per tree
    'random_state': 171,
    'n_jobs': -1
    'objective': 'mse'
    }

algo_base = LGBMRegressor(**tree_params)

In [ ]:
# produce dataframe of feature importances and plot results.
fi_df = train.fit_n_iterations(
    n=100,
    algo=algo_base,
    df=df_at,
    regressors=features,
    target=target,
    feat_imp=True,
    quantile_bool=False
)

fi_df.describe()

In [ ]:
fi_top = fi_df[fi_df['imp'] > fi_df['imp'].describe()['75%']].reset_index(drop=True)

## Recursive Feature Elimination  
Included this step, as initially ran a feature importance and retained an arbitary 200 features.  We removed correlated variables and now running RFE will tell us if there's no improvement after the nth variable.  RFE won't improve the model perfomance but it will simplify the interpretability of the model and maintenance cost associated with gathering and maintaining the code base. 

In [ ]:
# rfe_res will return a variable that avoids elimination in at least 1 of the n iterations.
# below we choose how many iterations (n) a variable needs to survive - giving more confidence that it's a 
# predictive variable
n = 10
rfe_res = train.rfe_n_iterations(n, algo_base, df_at, fi_top.cols, target)

In [ ]:
appearances = 3
print(f"""
Shortlisted features: {len(fi_top.cols)}
Features making at least one iteration of RFE: {len(ref_res)}
Features making {appearances} of {n} appearances in the RFE iterations: {len(rfe_res[rfe_res > appearances])}
"""
)

In [ ]:
ref_shortlist = (rfe_res[rfe_res > (n / 2)]).index
np.sort(ref_shortlist)

In [ ]:
final_shortlist = np.unique(ref_shortlist.tolist() + ['drivetime'])
len(final_shortlist)

In [ ]:
X_trn, X_val, y_trn, y_val = (train._create_postcode_area_train_test_split(
    df_at,
    final_shortlist,
    target,
    171
))

## Plotting Partial Dependence Plots  
PDP recap;  
* Overwrite the selected feature so that each observation has the same value and predict the target variable.  Repeat this for each value observed for that feature.  
* Eg/ overwrite the drivetime to 10 for all postcodes, make a prediction.  Then overwrite the drivetimes to 11 for all postcodes, make a prediction... 12, 13, ... n.
* Now we can isolate the effect of this one variable, all other variables being equal.

### For the top 10 features

In [ ]:
top_10 = viz.feature_importances_df(X_trn, algo_base)[-10:]
top_10

In [ ]:
viz.plot_partial_dependence(
    algo_base,
    X_trn,
    list(top_10['feature']),
    figsize=(12, 8)
    )

## SHAP Values  
* Variables are ranked by feature importance  
* Each dot is an observation  
* The horizontal axis shows whether the value of the observation has a positive or negative effect on the prediction.  
* The colour shows whether the variables value is high (red) or low (blue)  
* observations centred around zero confirm that the variable is adding little value.

In [ ]:
viz.shap_vis(
    X_trn, algo_base,
    interaction_plot=False,
    dependence_plot=False,
    max_display=True
    )

## Test different algorithms

In [ ]:
X_trn, X_val, y_trn, y_val = (train._create_postcode_area_train_test_split(
    df_at,
    final_shortlist,
    target,
    171
))

In [ ]:
lazy = LazyRegressor(predictions=True)

model_res, _ = lazy.fit(X_trn, X_val, y_trn, y_val)
model_res[:10]

## Gridsearch Parameters  
This approach looks at hyper-parameters in logical pairs.  i.e assess the hyper-parameters controlling tree depth such as max depth and minium  number of observations per leaf node.  

* `max_depth`: maximum depth of each tree.  
* `min_child_weight` (xgboost) or `min_samples_leaf` (sklearn): minimum number of observations per leaf node.  
* `n_estimators`: number of decision trees.  
* `subsample`: proportion of samples used per tree.  
* `colsample_bytree`: proportion of features used per tree.  

Regularisation Parameters  
* `learning_rate`: values less than 1 make fewer corrections in the sequential gradient boosting and protect from overfitting.  
* `gamma`: min loss reduction required to make a split, higher value = fewer splits.  
* `alpha`: L1 regularisation - higher val = more regularisation  
* `lambda_`: L2 regularisation - smoother than L1... what does this mean?!  

The scoring metric is the negative median absolute error to follow the convention that higher values are better, thus the best_param attribute is still relevant!

In [ ]:
tree_params = {
    'random_state': 171,
    'n_jobs': -1,
    'objective': 'mse',
    'n_estimators': 2000
    }

winning_algo = LGBMRegressor(**tree_params)

### Create training, val and test sets  
We will use the validation set to tune the hyper-parameters, so it's necessary to create an additional test set of unseen data.

In [ ]:
X_trn, X_val, X_test, y_trn, y_val, y_test = (train._create_postcode_area_train_test_split(
    df_at,
    final_shortlist,
    target,
    171,
    test_size=0.15,
    test_set=True
))

### Get Base prediction (without tuning)

In [ ]:
# fit with eval set
eval_set = [(X_trn, y_trn), (X_val, y_val)]

winning_algo.fit(
    X_trn,
    y_trn,
    eval_metric='mse',
    eval_set=eval_set,
    early_stopping_rounds=10,  # if lightgbm include this line.
    # if using early stopping set verbose to 50, else False.
    verbose=50
)

In [ ]:
tree_params['n_estimators'] = 175 # set this based on the verbose above.
winning_algo = LGBMRegressor(**tree_params)
base_model = winning_algo.fit(X_trn, y_trn)

In [ ]:
# check scores across training, val and test sets for overfitting.
train.train_val_test_scores(
    base_model,
    '_rmse',
    df_at,
    final_shortlist,
    target,
    171,
    test_set=True,
    test_sise=0.25
)

In [ ]:
n_iter = 10
train.fit_n_iterations(
    n_iter,
    base_model,
    df_at,
    final_shortlist,
    target,
    feat_imp=False,
    quantile_bool=False
)

returns...  
Avg score over 10 samples (one standard deviation in brackets)  
R2: 88 (87 - 89%)  
MAE: £70k (£65-72k)  
Mean APE: 139% (100 - 175%)  
Median APE: 28% (27 - 30%)

In [ ]:
# reset
tree_params['n_estimators']  = 100
winning_algo = LGBMRegressor(**tree_params)

### Parameters controlling the architecture/ depth of trees

In [ ]:
max_depth = [-1]
min_child_weight = [int(x) for x in np.linspace(start=20, stop=300, num=5)]  # may need to widen this initially to search the right space.

param_grid - {
    'max_depth': max_depth,
    'min_child_weight': min_child_weight  # min_samples_leaf if sklearn algo.
}

print(train.check_gridsearch_iter(param_grid))  # checks the number of parameters searched to warn if enormous!

In [ ]:
# run gridsearch.
gs_results, best_params, best_algo1 = train.fit_gridsearch_plot_results(
    randomgrid=False,
    estimator=winning_algo,
    scoring_metric='neg_mean_squared_error',
    param_grid=param_grid,
    X=X_trn,
    y=y_trn
)

^^^ Returns...  
Best score: xxxx  
Best params: {'max_depth': x, 'min_child_weight': y}

Then a plot of each parameter vs corresponding effect on neg_mean_squared_error.

In [ ]:
# check scores across training, val and test sets for overfitting.
train.train_val_test_scores(...)

### Paramters controlling the sampling of data

In [ ]:
subsample = [1 / 10.0 for i in range(6, 11)]
colsample_bytree = [1 / 10.0 for i in range(6, 10)]
subsample_freq = [2, 4, 8, 12, 14]

param_grid = {
    'subsample': subsample,
    'colsample_bytree': colsample_bytree,
    'subsample_freq': subsample_freq
}
print(train.check_gridsearch_iter(param_grid))

In [ ]:
# run gridsearch.
gs_results, best_params, best_algo2 = train.fit_gridsearch_plot_results(
    randomgrid=False,
    estimator=winning_algo,
    scoring_metric='neg_mean_squared_error',
    param_grid=param_grid,
    X=X_trn,
    y=y_trn
)

# check scores across training, val and test sets for overfitting.
train.train_val_test_scores(...)

### Search learning rate

In [ ]:
learning_rate = [x.round(2) for x in np.linspace(start=0.01, stop=0.2, num=5)]
param_grid = {
    'learning_rate': learning_rate,
    'n_estimators': [2000]  # increase number of trees to see effect of lower learning rate/ higher trees combo.
}
param_grid

In [ ]:
# run gridsearch.
gs_results, best_params, best_algo3 = train.fit_gridsearch_plot_results(
    randomgrid=False,
    estimator=winning_algo,
    scoring_metric='neg_mean_squared_error',
    param_grid=param_grid,
    X=X_trn,
    y=y_trn
)

# check scores across training, val and test sets for overfitting.
train.train_val_test_scores(...)

### Search Trees  

nb use a gridsearch here if its Random Forest, for XGBoost because it's sequentially learning we plot the improvement across the training and validation set by passing the optional `eval_set`.  To start set a high number of trees and analyse the plot for the point of inflection; the point with the validation score plateaus/ worsens whilst the training score continues to improve/ overfit.

In [ ]:
eval_set = [(X_trn, y_trn), (X_val, y_val)]

best_algo3.fit(
    X_trn,
    y_trn,
    eval_set=eval_set,
    early_stopping_rounds=10,
    verbose=50
)

train._rmse(y_test, best_algo3.predict(X_test))

In [ ]:
# I think this code could be used with XGBoost...
# which would plot the training & validation scores over n_estsimators;

# train.plot_early_stopping(best_algo3, xlim=2000, ylim=35000)

In [ ]:
# set the number of trees from the verbose;
best_params = best_algo3.get_params()
best_params['n_estimators'] = 1723 

### Fit confidence interval models...  
Here the model's objective is switched to `quantile` so that the algorithm trains to minimise the error around the given quantiles instead of the mean error.

In [ ]:
mod_dict = {}
# save MDE model as the 50th percentile (this was a cheat that meant we didn't need to change the GCS folder structure.)

model = LGBMRegressor(**best_params).fit(X_trn, y_trn)
mod_dict[50] = model

# switch the objective
best_params['objective'] = 'quantile'
quantiles = ['.05', '.3', '.7', '.95']

for q in qunatiles:
    model = LGBMRegressor(alpha=q, **best_params).fit(X_trn, y_trn)
    mod_dict[np.int(q * 100)] = model

### Final Model Evaluation Metrics  
Fit on multiple test/train splits to check confidence in the model.  
* $R^2$ : The proportion of the variance for a dependent variables that's explained by independent variables.  
* Mean Absolute Error: Average magnitude of errors without considering direction and each observation has the same weight.  
* Root Mean Squared Error: Also measures the average magnitude of the error, commonly described as the standard deviation of the residuals.  By squaring before taking the mean results in larger errors being penalised more, this RMSE is always equal to or larger than MAE.  This can be helpful if being £10k out is more than twice as bad as being £5k out.  But if it;s just twice as bad then MAE is more appropriate.  
* Mean Absolute Percentage Error: can only be calculated on observations with existing sales/ target variable to avoid "devide by zero" error.

In [ ]:
# check scores across train, val and test sets...
train.train_val_test_scores(...)

In [ ]:
# this plot showed a scatter plot of the observations true value vs predicted.  
# therefore if the prediction was completely accurate then you'd have a diagnal line plotted, deviation from this shows the error.
viz.plot_regression(
    mod_dict[50],
    mod_dict[5],
    mod_dict[95],
    df_at,
    final_shortlist,
    target,
    test_set=False
    )

## Pickle model and save to GCS

**Set model id** 
Using the naming convention {target}_{training_data_year}_{scenario}, eg/ "wtr_online_sales_TY19_base".

In [ ]:
gcs = utils.read_config('gcs_paths.yaml')
from src import version_name
model_id = f"{target}_FY19_base"
model_path = f"{version_name}_/{gcs['model_output_path']}/{model_id}"

In [ ]:
# save model
for mod in mod_dict:
    file_name = f"{target}_FY19_{mod}_base.pickle"
    write.pickle_to_gcs(mod_dict[mod], model_path, file_name)

In [ ]:
# save meta data
write.save_model_details(X_trn, mod_dict[50], model_path)